## Purpose 



 
This test case was created to validate the direct-forcing immersed boundary method implemented (DF) in TrioCFD by comparing it to the ALE discretization using the SPHERIC benchmark [1]. The geometry is taken from [1], and the test case with a Reynolds number of 50 was selected.


In this test, the flow around a horizontally moving square in a tank is investigated. Initially, the square is stationary and reaches its maximum velocity within one second.

Validation made by : Amal DERRO
 
Report generated  19/05/2025

In [ ]:
from trustutils import run
from multiprocessing import Pool

def my_run_cases(data_folder,test_case, nb_proc):
    run.addCase(data_folder,test_case, nbProcs=nb_proc)
    run.printCases()
    run.runCases()

def fonction_wrapper(args):
    return my_run_cases(*args)  # Décompresser les arguments (tuple)

if __name__ == "__main__":
    params = [("./VEF","square.data" , 2), ("./VDF","square.data", 12), ("./ALE","SquareObstacle_ALE.data",1)]  # Liste des tuples de paramètres
    num_processes = 3  # Nombre de processus à utiliser

    # Créer un Pool pour exécuter la fonction en parallèle
    with Pool(processes=num_processes) as pool:
        results = pool.map(fonction_wrapper, params)  # Exécute la fonction avec chaque paire de paramètres

## Problem Description 
 The geometry is described in Figure 1. The red line represents the solid boundary of the tank, while the black line indicates the initial position of the square at time t=0. For t>0, the square moves to the right with a prescribed motion, starting from rest and accelerating to a final steady maximum velocity (see Boundary Conditions).

### Geometry


In [ ]:
from IPython.display import Image, display

# Afficher l'image
display(Image(filename='src/geometry.png'))




######                       Figure 1: Geometry used in the simulation


### Initial Conditions and Boundary Conditions 


The square is moving and it reaches its maximum velocity within one second, as ploted below.

In [ ]:
from IPython.display import Image, display

# Afficher l'image
display(Image(filename='src/velocity.png'))

######                       Figure 2: Square velocity

The boundaries of the cavity are assigned to the fixed wall, except for the right one, which is assigned to a null imposed pressure.

### Fluid Properties 


The square is immersed in a Newtonian incompressible homogeneous fluid, with mass density $\rho \,(1 \,Kg/m^3 )$ and kinematic viscosity $\nu \,(0.02\, m^2/s)$.


### Model Options 


The fluid problem with moving boundaries is handled using the direct-forcing immersed boundary method, with linear interpolation for VDF discretization and linear basis functions for VEF discretization.


In the IBM approach, the computational domain includes both the fluid and solid domains. The interface is described using a Lagrangian mesh, and interface tracking is performed through the Front-Tracking technique.

## Results 
### Validation Specific Informations 
* Version TRUST : 1.9.5 
* Problem : Probleme\_FT\_Disc\_gen
* Dimension : 2D
* Time scheme : Scheme\_euler\_explicit
* Medium : Fluide\_Incompressible
* Hydraulic problem : Navier\_Stokes\_FT\_Disc
* Interface tracking : Transport\_Interfaces\_FT\_Disc
* Convection scheme: amont
* Generated Test cases : 
	- ./square.data :  


### Plot Data 
 


Probes: 1D Plots


The drag coefficient pressure component $C_{D,p}$ (pressure component) is given by:



$\displaystyle{C_d = \frac{\int_{\text{interface}} (-p \cdot \vec{n}_x) \, dS}{\frac{1}{2} \cdot \rho  \cdot S \cdot U_{\text{max}}^2}}$

where $U_{\text{max} = 1 \, \text{m/s}}$ (the maximum velocity of the obstacle). This value is plotted for the described test case (IBM) and compared to ALE_TrioCFD using a amont convection scheme and an explicit Euler time scheme.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Paramètre
vitesse=1
surface=1
rho=1
param = 1/(0.5*(vitesse**2)*surface*rho)

frequence = 5

# Lecture du fichier IBM
data_ibm_VDF = np.loadtxt('build/VDF/square_Friction_Pression_sur_square.out')
x_ibm_VDF = data_ibm_VDF[::frequence, 0]
y_ibm_VDF = param * np.abs(data_ibm_VDF[::frequence, 1])

data_ibm_VEF = np.loadtxt('build/VEF/square_Friction_Pression_sur_square.out')
x_ibm_VEF = data_ibm_VEF[::frequence, 0]
y_ibm_VEF = param * np.abs(data_ibm_VEF[::frequence, 1])

# Lecture du fichier ALE
data_ale = np.loadtxt('build/ALE/SquareObstacle_ALE_SONDE_DRAG_COEF_PRESSURE_COMP.son')
x_ale = data_ale[::frequence, 0]
y_ale = data_ale[::frequence, 1]

# Tracé
plt.figure(figsize=(10, 5))
plt.plot(x_ibm_VEF, y_ibm_VEF, 'o-', label='IBM-VEF')
plt.plot(x_ibm_VDF, y_ibm_VDF, '^-', label='IBM-VDF')

plt.plot(x_ale, y_ale, 's-', label='ALE')
plt.xlabel('t')
plt.ylabel('Cd (pressure component)')
plt.title('Drag coefficient pressure component')
plt.legend()
plt.grid(True)
plt.show()




The drag coefficient viscous component $\displaystyle{C_{D,\nu}}$ (viscous component) is given by:


$\displaystyle{C_{D,\nu} = \frac{\int_{\text{interface}} \left( \boldsymbol{\tau} \cdot \vec{n} \right)_x \, dS}{\frac{1}{2} \cdot \rho \cdot S \cdot U_{\text{max}}^2} }$


where $\displaystyle{\boldsymbol{\tau}}$ is the viscous stress tensor and $\displaystyle{\left( \boldsymbol{\tau} \cdot \vec{n} \right)_x}$ is its projection in the flow direction (x). As before, $\displaystyle{U_{\text{max}} = 1 \, \text{m/s}}$ is the reference velocity. This value is computed for the IBM test case and compared to ALE_TrioCFD with an upwind convection scheme and explicit Euler time integration.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Paramètre
vitesse=1
surface=1
rho=1
param = 1/(0.5*(vitesse**2)*surface*rho)

frequence = 5

# Lecture du fichier IBM
data_ibm_VDF = np.loadtxt('build/VDF/square_Friction_conv_diff_sur_square.out')
x_ibm_VDF = data_ibm_VDF[::frequence, 0]
y_ibm_VDF = param * np.abs(data_ibm_VDF[::frequence, 1])

data_ibm_VEF = np.loadtxt('build/VEF/square_Friction_conv_diff_sur_square.out')
x_ibm_VEF = data_ibm_VEF[::frequence, 0]
y_ibm_VEF = param * np.abs(data_ibm_VEF[::frequence, 1])

# Lecture du fichier ALE
data_ale = np.loadtxt('build/ALE/SquareObstacle_ALE_pb_Contrainte_visqueuse.out')
x_ale = data_ale[::frequence, 0]
y_ale = data_ale[::frequence, 7]+data_ale[::frequence, 9]
y_ale =param *abs(y_ale)

# Tracé
plt.figure(figsize=(10, 5))
plt.plot(x_ibm_VEF, y_ibm_VEF, 'o-', label='IBM-VEF')
plt.plot(x_ibm_VDF, y_ibm_VDF, '^-', label='IBM-VDF')
plt.plot(x_ale, y_ale, 's-', label='ALE')
plt.xlabel('t')
plt.ylabel('Cd (viscous component)')
plt.title('Drag coefficient viscous component')
plt.legend()
plt.grid(True)
plt.show()

Below, we present several 1D plots showing the evolution of pressure and velocity along different segments, with the aim of comparing the results obtained using the IBM and ALE methods.

In [ ]:
from trustutils import plot 
 
fig = plot.Graph(r"Pressure along line (Y=1.5) ") 
fig.addSegment("ALE/SquareObstacle_ALE_SONDE_PRESSION_SEGMENT_H1.son",label=r"ALE",marker='-x') 
fig.addSegment("VDF/square_SONDE_PRESSION_SEGMENT_H1.son",label=r"IBM_VDF",marker='-^') 
fig.addSegment("VEF/square_SONDE_PRESSION_SEGMENT_H1.son",label=r"IBM_VEF",marker='-o') 
fig.label(r"X",r"Pressure")




In [ ]:
from trustutils import plot 
 
fig = plot.Graph(r"Pressure along line (Y=3.5) ") 
fig.addSegment("ALE/SquareObstacle_ALE_SONDE_PRESSION_SEGMENT_H3.son",label=r"ALE",marker='-x') 
fig.addSegment("VDF/square_SONDE_PRESSION_SEGMENT_H3.son",label=r"IBM_VDF",marker='-^') 
fig.addSegment("VEF/square_SONDE_PRESSION_SEGMENT_H3.son",label=r"IBM_VEF",marker='-o')
fig.label(r"X",r"Pressure")

In [ ]:
from trustutils import plot 
 
fig = plot.Graph(r"Velocity along line Y=1.5 ") 
fig.addSegment("ALE/SquareObstacle_ALE_SONDE_VITESSEX_SEGMENT_H1.son",label=r"ALE",marker='-x') 
fig.addSegment("VDF/square_SONDE_VITESSEX_SEGMENT_H1.son",label=r"IBM_VDF",marker='-^') 
fig.addSegment("VEF/square_SONDE_VITESSEX_SEGMENT_H1.son",label=r"IBM_VEF",marker='-o') 
fig.label(r"X",r"Vx")

In [ ]:
from trustutils import plot 
 
fig = plot.Graph(r"Velocity along line Y=1.5 ") 
fig.addSegment("ALE/SquareObstacle_ALE_SONDE_VITESSEY_SEGMENT_H1.son",label=r"ALE",marker='-x') 
fig.addSegment("VDF/square_SONDE_VITESSEY_SEGMENT_H1.son",label=r"IBM_VDF",marker='-^') 
fig.addSegment("VEF/square_SONDE_VITESSEY_SEGMENT_H1.son",label=r"IBM_VEF",marker='-o') 
fig.label(r"X",r"VY")

Probes: 3D Plots


# Visualisation of contours
 
 

### Velocity

#### IBM_VDF

In [ ]:
from trustutils import visit
 
visu = visit.Show("./VDF/IBM.lata","Pseudocolor","VITESSE_ELEM_DOM_magnitude",plotmesh=False)


visu.plot()

#### ALE

In [ ]:
from trustutils import visit
 
visu = visit.Show("./ALE/lata/ALE.lata","Pseudocolor","VITESSE_ELEM_dom_magnitude",plotmesh=False)


visu.plot()

#### IBM_VEF

In [ ]:
from trustutils import visit
 
visu = visit.Show("./VEF/IBM.lata","Pseudocolor","VITESSE_ELEM_DOM_magnitude",plotmesh=False)


visu.plot()

### Pressure

#### IBM_VDF

In [ ]:
from trustutils import visit
 
visu = visit.Show("./VDF/IBM.lata","Pseudocolor","PRESSION_SOM_DOM",plotmesh=False)

visu.plot()

#### ALE

In [ ]:
from trustutils import visit
 
visu = visit.Show("./ALE/lata/ALE.lata","Pseudocolor","PRESSION_SOM_dom",plotmesh=False)

visu.plot()

#### IBM_VEF

In [ ]:
from trustutils import visit
 
visu = visit.Show("./VEF/IBM.lata","Pseudocolor","PRESSION_SOM_DOM",plotmesh=False)

visu.plot()

## Conclusion 
The analysis of the plots of various quantities such as aerodynamic coefficients, velocities, and pressures shows good agreement between the results obtained using the DF-IBM and ALE methods. This validates the implementation of the DF-IBM approach in the context of the SPHERIC benchmark test cases [1].



## References: 

* 1. A. Colagrossi, 'SPHERIC benchmark 6: 2D Incompressible flow around a moving square inside a rectangular box. - 2D SPH Validation.', SPHERIC, [Online]. Available: http://spheric-sph.org/tests/test-6. [Accessed Sept. 29, 2017].

